In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score
from nltk.stem import PorterStemmer 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import gensim 
from gensim.models import Word2Vec 
from nltk.probability import FreqDist
from nltk.corpus import gutenberg
from multiprocessing import Pool
from scipy import spatial
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler , OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

C:\Users\Albert\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
feature = [["This was a great movie with a good cast, all of them hitting on allcylinders.",
            "Even if you're a huge Sandler fan, please don't bother with thisextremely disappointing comedy!",
            "A movie of outstanding brilliance and a poignant and unusual love story",
            "I had the misfortune to watch this rubbish on Sky Cinema Max in a coldwinter night",
            "I am at a distinct disadvantage here. I have not seen the first two moviesin this series",
            "This program is a lot of fun and the title song is so catchy I can't get itout of my head"]]
#mengubah tipe data feature dan target dari object ke string
feature = str(feature)
target = str(feature)

# Case Folding

Di bagian ini akan dilakukan case folding terhadap feature berupa membuat setiap huruf menjadi lower case dan membuang tanda tanda baca  

In [3]:
#Lower
#digunakan untuk melowercase setiap character dalam string string, 
feature = feature.lower()
print(feature)

[['this was a great movie with a good cast, all of them hitting on allcylinders.', "even if you're a huge sandler fan, please don't bother with thisextremely disappointing comedy!", 'a movie of outstanding brilliance and a poignant and unusual love story', 'i had the misfortune to watch this rubbish on sky cinema max in a coldwinter night', 'i am at a distinct disadvantage here. i have not seen the first two moviesin this series', "this program is a lot of fun and the title song is so catchy i can't get itout of my head"]]


In [4]:
#replace
#digunakan untuk mengereplace character character yang tidak di inginkan di dalam string
feature = feature.replace(".","")
feature = feature.replace("?","")
feature = feature.replace("'","")
feature = feature.replace(",","")
feature = feature.replace("!","")
feature = feature.replace("``","")
feature = feature.replace(")","")
feature = feature.replace("''","")
feature = feature.replace('"','')
feature = feature.replace('(','')
feature = feature.replace('[','')
feature = feature.replace(']','')
print(feature)

this was a great movie with a good cast all of them hitting on allcylinders even if youre a huge sandler fan please dont bother with thisextremely disappointing comedy a movie of outstanding brilliance and a poignant and unusual love story i had the misfortune to watch this rubbish on sky cinema max in a coldwinter night i am at a distinct disadvantage here i have not seen the first two moviesin this series this program is a lot of fun and the title song is so catchy i cant get itout of my head


Tokenization

di bagian ini akan dilakukan tokenization. Tokenization berfungsi untuk memisahkan setiap kata,tanda baca, simbol dan entitas
dalam sebuah kalimat. nantinya hasil pemisahan ini yang di sebut token akan digunakan untuk analisa nantinya.

In [5]:
#memisahkan setiap kata,simbol,tanda baca dan lain lain
featureword = nltk.tokenize.word_tokenize(feature)
print(featureword)

['this', 'was', 'a', 'great', 'movie', 'with', 'a', 'good', 'cast', 'all', 'of', 'them', 'hitting', 'on', 'allcylinders', 'even', 'if', 'youre', 'a', 'huge', 'sandler', 'fan', 'please', 'dont', 'bother', 'with', 'thisextremely', 'disappointing', 'comedy', 'a', 'movie', 'of', 'outstanding', 'brilliance', 'and', 'a', 'poignant', 'and', 'unusual', 'love', 'story', 'i', 'had', 'the', 'misfortune', 'to', 'watch', 'this', 'rubbish', 'on', 'sky', 'cinema', 'max', 'in', 'a', 'coldwinter', 'night', 'i', 'am', 'at', 'a', 'distinct', 'disadvantage', 'here', 'i', 'have', 'not', 'seen', 'the', 'first', 'two', 'moviesin', 'this', 'series', 'this', 'program', 'is', 'a', 'lot', 'of', 'fun', 'and', 'the', 'title', 'song', 'is', 'so', 'catchy', 'i', 'cant', 'get', 'itout', 'of', 'my', 'head']


In [6]:
#memunculkan frekeunsi keluarnya kata
frequency = nltk.FreqDist(featureword)
print(frequency.most_common())

[('a', 8), ('this', 4), ('of', 4), ('i', 4), ('and', 3), ('the', 3), ('movie', 2), ('with', 2), ('on', 2), ('is', 2), ('was', 1), ('great', 1), ('good', 1), ('cast', 1), ('all', 1), ('them', 1), ('hitting', 1), ('allcylinders', 1), ('even', 1), ('if', 1), ('youre', 1), ('huge', 1), ('sandler', 1), ('fan', 1), ('please', 1), ('dont', 1), ('bother', 1), ('thisextremely', 1), ('disappointing', 1), ('comedy', 1), ('outstanding', 1), ('brilliance', 1), ('poignant', 1), ('unusual', 1), ('love', 1), ('story', 1), ('had', 1), ('misfortune', 1), ('to', 1), ('watch', 1), ('rubbish', 1), ('sky', 1), ('cinema', 1), ('max', 1), ('in', 1), ('coldwinter', 1), ('night', 1), ('am', 1), ('at', 1), ('distinct', 1), ('disadvantage', 1), ('here', 1), ('have', 1), ('not', 1), ('seen', 1), ('first', 1), ('two', 1), ('moviesin', 1), ('series', 1), ('program', 1), ('lot', 1), ('fun', 1), ('title', 1), ('song', 1), ('so', 1), ('catchy', 1), ('cant', 1), ('get', 1), ('itout', 1), ('my', 1), ('head', 1)]


In [7]:
#memisahkan setiap kalimat dalam feature feature
featurekalimat = nltk.tokenize.sent_tokenize(feature)
print(featurekalimat)

['this was a great movie with a good cast all of them hitting on allcylinders even if youre a huge sandler fan please dont bother with thisextremely disappointing comedy a movie of outstanding brilliance and a poignant and unusual love story i had the misfortune to watch this rubbish on sky cinema max in a coldwinter night i am at a distinct disadvantage here i have not seen the first two moviesin this series this program is a lot of fun and the title song is so catchy i cant get itout of my head']


Filtering

Di bagian ini akan di lakukan filtering. Filtering dalam hal ini di bagi menjadi 2 yakni stoplist dan wordlist. stoplist adalah dimana kita membuang kata kata tidak penting seperti a,with yang merupakan stopword dalam bahasa inggris.sedangkan untuk wordlist adalah dimana kita menyimpan kata kata penting yang kita perlukan.

In [8]:
#dibagian ini dilakukan stopword removal untuk membuang stopword stopword seperti a, with yang dapat menganggu training model kita
Stopwords =  set(stopwords.words('english'))
 
featureremoval = []
for i in featureword:
    if i not in Stopwords:
        featureremoval.append(i)
 
print(featureremoval)

['great', 'movie', 'good', 'cast', 'hitting', 'allcylinders', 'even', 'youre', 'huge', 'sandler', 'fan', 'please', 'dont', 'bother', 'thisextremely', 'disappointing', 'comedy', 'movie', 'outstanding', 'brilliance', 'poignant', 'unusual', 'love', 'story', 'misfortune', 'watch', 'rubbish', 'sky', 'cinema', 'max', 'coldwinter', 'night', 'distinct', 'disadvantage', 'seen', 'first', 'two', 'moviesin', 'series', 'program', 'lot', 'fun', 'title', 'song', 'catchy', 'cant', 'get', 'itout', 'head']


Stemming

Di bagian ini akan dilakukan stemming. stemming adalah suatu proses dimana kita membuang infeksi kata untuk menjadikan kata terseut menjadi kata dasarnya. Walaupun kata dengan infeksi tersebut tidak memiliki arti yang sama dengan kata dasarnya. Hal ini dilakukan untuk memperlancar model untuk melakukan training

In [9]:
ps = PorterStemmer() 
for k in featureremoval: 
    print(k, "=", ps.stem(k))

great = great
movie = movi
good = good
cast = cast
hitting = hit
allcylinders = allcylind
even = even
youre = your
huge = huge
sandler = sandler
fan = fan
please = pleas
dont = dont
bother = bother
thisextremely = thisextrem
disappointing = disappoint
comedy = comedi
movie = movi
outstanding = outstand
brilliance = brillianc
poignant = poignant
unusual = unusu
love = love
story = stori
misfortune = misfortun
watch = watch
rubbish = rubbish
sky = sky
cinema = cinema
max = max
coldwinter = coldwint
night = night
distinct = distinct
disadvantage = disadvantag
seen = seen
first = first
two = two
moviesin = moviesin
series = seri
program = program
lot = lot
fun = fun
title = titl
song = song
catchy = catchi
cant = cant
get = get
itout = itout
head = head


# Word Embeeding

Word embeeding adalah suatu proses dimana kita mengkonversi sebuah teks menjadi angka dalam bentuk vektor. hal ini dilakukan karena sebagian besar algoritma dalam python tidak memiliki kemampuan untuk melakukan training terhadap string dan text. pada soal ini telah kita lakukan casefolding menjadi kedalam bentuk kata kata dan kemudian kata kata tersebut akan di konversi kedalam bentuk numerik vektor 

In [34]:
#disini saya kembali mendeclare setiap kata kata dikarenakan jika saya langsung menggunakan featureremoval dari bagian stemming maka lenghtnya akan menjadi sebanyak kata kata bukan sebanyak kalimat yang ada
# featurecasefold = np.array([ ['great', 'movi', 'good', 'cast', 'hit','cylind'],
#                              ['even', 'your', 'huge', 'sandler', 'fan', 'pleas', 'dont', 'bother', 'extreme', 'disappoint', 'comedi'],
#                              ['movi', 'outstand', 'brillianc', 'poignant', 'unusu', 'love', 'stori'],
#                              ['misfortun', 'watch', 'rubbish', 'sky', 'cinema', 'max', 'coldwint', 'night'],
#                              ['distinct', 'disadvantag', 'seen', 'first', 'two', 'movi', 'seri'],
#                              ['program', 'lot', 'fun', 'titl', 'song', 'catchi', 'cant', 'get', 'head']])
featurecasefold = np.array([ ['great movi good cast hit cylind'],
                             ['even your huge sandler fan pleas dont bother extreme disappoint comedi'],
                             ['movi outstand brillianc poignant unusu love stori'],
                             ['misfortun watch rubbish sky cinema max coldwint night'],
                             ['distinct disadvantag seen first two movi seri'],
                             ['program lot fun titl song catchi cant get head']])
target = np.array([ ["Positive"],
          ["Negative"],
          ["Positive"],
          ["Negative"],
          ["Negative"],
          ["Positive"] ])
# featurecasefold = np.array(featurecasefold)
# target = np.array(target)

In [35]:
#print tipe dari feature dan banyak featurenya
print('Type: ', type(featurecasefold))
print('Length: ', len(featurecasefold))
print('Type: ', type(target))
print('Length: ', len(target))


Type:  <class 'numpy.ndarray'>
Length:  6
Type:  <class 'numpy.ndarray'>
Length:  6


In [36]:
#model implementation
#model ini akan menggunakan hierarchical softmax untuk mengtrain data akan memiliki ukuran feature sebesar 100. ini membuat model ini akan memiliki hierarchial output dan menggunakan softmax function pada layer terakhirnya
modelfeature = Word2Vec(sentences = featurecasefold, size = 100, sg = 1, window = 3, min_count = 1, iter = 10, workers = Pool()._processes)
modelfeature.init_sims(replace = True)
modelfeature.save('word2vecmodelfeature')
modelfeature = Word2Vec.load('word2vecmodelfeature')
print(modelfeature)
modeltarget = Word2Vec(sentences = target, size = 100, sg = 1, window = 3, min_count = 1, iter = 10, workers = Pool()._processes)
modeltarget.init_sims(replace = True)
modeltarget.save('word2vecmodeltarget')
modeltarget = Word2Vec.load('word2vecmodeltarget')
print(modeltarget)
#print vocab yang ada di bawah berguna untuk mengetahui beragam macam string yang ada, seperti contohnya untuk target hanya ada 2 macam yaitu positive dan negative maka dari itu vocabnya hanya ada 2

Word2Vec(vocab=6, size=100, alpha=0.025)
Word2Vec(vocab=2, size=100, alpha=0.025)


In [37]:
#Vektor yang diprint digunakan untuk mempresentasikan setiap kata kedalam bentuk vektornya
featurenew = modelfeature[modelfeature.wv.vocab]
print(featurenew)
#normalize
Scaler = MinMaxScaler()
featurenew = Scaler.fit_transform(featurenew)
#disini data di transpose agar kolom dari feature dan target sama
featurenew = featurenew.transpose()
featurenew.shape

[[-0.14139578 -0.15452746 -0.10391676  0.00444754 -0.02853733 -0.10024175
  -0.07323908 -0.0310267   0.08809736  0.02405438 -0.002307    0.02654263
  -0.02779163 -0.00429139  0.10922731  0.10282768 -0.06354536 -0.11234605
  -0.10737465 -0.04549555 -0.02415729  0.05853832  0.1649285  -0.02536117
  -0.05525004 -0.0293949  -0.15411346  0.1519554   0.02964898 -0.16430447
   0.0867385   0.00454118 -0.11707285  0.04211499 -0.04840746  0.01044454
  -0.17067054  0.02813452 -0.05965364 -0.18040541  0.04051857 -0.05972388
  -0.00956252  0.01608486  0.08258258 -0.16327219 -0.14363325 -0.00535721
   0.1401946  -0.18016003  0.00566673 -0.16348538 -0.09900035  0.12174911
   0.01890818 -0.14724083 -0.08860787 -0.03583032 -0.05488735 -0.04508565
  -0.08790813 -0.11051644  0.06770249 -0.01351312 -0.13169903 -0.04783081
   0.04022935 -0.17644376 -0.17518418 -0.06456487 -0.07130415 -0.06696139
  -0.09849687  0.08353431 -0.04065412  0.16618454 -0.04925666  0.02939223
  -0.15605785 -0.09468408 -0.12717703 

C:\Users\Albert\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(100, 6)

In [38]:
#Vektor untuk target yang hanya terdiri dari 2 vocab yakni positif dan negatif
targetnew = modeltarget[modeltarget.wv.vocab]
#normalize
Scaler = MinMaxScaler()
targetnew = Scaler.fit_transform(targetnew)
#disini data di tranpose agar kolom dari feature dan target sama
targetnew = targetnew.transpose()
targetnew.shape

C:\Users\Albert\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(100, 2)

In [42]:
#mengubah tf dari versi 2 ke versi 1
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# featurecasefold = featurecasefold.reshape(-1,1)
# target = target.reshape(-1,1)

# modelfeature = modelfeature.reshape(1,1)
# modeltarget = modeltarget.reshape(1,1)

#input adalah feature yang telah di word embeeding sebelumnya dengan nama featurenew
#outputnya adalah target yang sebelumnya telah di word embeeding dengan nama targetnew

#generate layer,weight and bias
layer = {
    'input' : 6,
    'hidden' : 3,
    'output' : 2
}
#init weight and bias random
weight = {
    'input_to_hidden' : tf.Variable(tf.random_normal([layer['input'], layer['hidden']])),
    'hidden_to_output' : tf.Variable(tf.random_normal([layer['hidden'], layer['output']])) 
}
bias = {
    'input_to_hidden' : tf.Variable(tf.random_normal([layer['hidden']])),
    'hidden_to_output' : tf.Variable(tf.random_normal([layer['output']]))
}
#Untuk mempermudah feed_dict di bawah
input_placeholder = tf.placeholder(tf.float32, [None, layer['input']])
output_placeholder = tf.placeholder(tf.float32, [None, layer['output']])

#feed forward function
def feed_forward(featurenew):
    weightxbias1 = tf.matmul(featurenew, weight['input_to_hidden']) + bias['input_to_hidden']
    activation1 = tf.nn.sigmoid(weightxbias1)

    weightxbias2 = tf.matmul(activation1, weight['hidden_to_output']) + bias['hidden_to_output']
    activation2 = tf.nn.sigmoid(weightxbias2)

    return activation2
#generate learning rate and epoch
learning_rate = 0.3
epoch = 5000
#train dataset using mean squared error and gradient descent optimizer
output = feed_forward(input_placeholder)
mse = tf.reduce_mean(0.5 * (output - output_placeholder) ** 2) 
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(mse)
saver = tf.train.Saver()
feature_train, feature_test, target_train, target_test = train_test_split(featurenew,targetnew,test_size=0.2)

#print(feature_test)
#data training
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
#declare feature dan target untuk data training
    feed_train = {
        input_placeholder: feature_train,
        output_placeholder: target_train
    }
#mengprint error setiap 100 epoch
    for i in range(epoch):
        sess.run(train, feed_dict=feed_train)
        error = sess.run(mse, feed_dict = feed_train)
        #pred = sess.run(output, feed_dict = feed_train)
        if i % 100 == 0:
            print('Epoch: {} Error: {} '. format(i+100, error))
            #print('Pred : {}',pred)
#declare feature dan target untuk data testing                  
    feed_test = {
        input_placeholder: feature_test,
        output_placeholder: target_test
    }
#     for i in range(epoch):
#         sess.run(train, feed_dict=feed_test)
#         pred = sess.run(output, feed_dict = feed_test)
#         if i % 100 == 0:
#             print('Pred : {}',pred)
#accuracy,precision dan recall  
    matches = tf.equal(tf.argmax(output, axis=1),tf.argmax(output_placeholder,axis=1))
    conf  = tf.math.confusion_matrix(tf.argmax(output_placeholder, axis=1),tf.argmax(output,axis=1))
    print('Conf : {}'.format(sess.run(conf,feed_dict = feed_test)))
#     matrix = format(sess.run(conf,feed_dict = feed_test))
#     print(prediction)
#     print(matches)
    accuracy = tf.reduce_mean(tf.cast(matches,tf.float32))#coreect predicition / total prediction
    precision = conf[0,0] / (conf[0,0] + conf[0,1])#true positive/ true positive+false positive
    recall = conf[0,0] / (conf[0,0] + conf[1,0])#true positive/true positive + false negative
    #recall = tf.metrics.recall(tf.argmax(output_placeholder, axis=1),tf.argmax(output,axis=1))
   # precision = tf.metrics.precision(tf.argmax(input = output, axis=1),tf.argmax(output_placeholder,axis=1))
    print('Accuracy: {}'.format(sess.run(accuracy, feed_dict =feed_test)))
    print('recall: ', format(sess.run(recall,feed_dict = feed_test)))
    print('precision:',format(sess.run(precision,feed_dict = feed_test)))

Epoch: 100 Error: 0.15904735028743744 
Epoch: 200 Error: 0.1321330964565277 
Epoch: 300 Error: 0.12950745224952698 
Epoch: 400 Error: 0.1287432610988617 
Epoch: 500 Error: 0.12811943888664246 
Epoch: 600 Error: 0.12758156657218933 
Epoch: 700 Error: 0.12711532413959503 
Epoch: 800 Error: 0.12670888006687164 
Epoch: 900 Error: 0.12635202705860138 
Epoch: 1000 Error: 0.12603603303432465 
Epoch: 1100 Error: 0.12575359642505646 
Epoch: 1200 Error: 0.12549862265586853 
Epoch: 1300 Error: 0.12526610493659973 
Epoch: 1400 Error: 0.12505188584327698 
Epoch: 1500 Error: 0.12485256046056747 
Epoch: 1600 Error: 0.1246652752161026 
Epoch: 1700 Error: 0.12448771297931671 
Epoch: 1800 Error: 0.12431792169809341 
Epoch: 1900 Error: 0.12415430694818497 
Epoch: 2000 Error: 0.12399550527334213 
Epoch: 2100 Error: 0.12384037673473358 
Epoch: 2200 Error: 0.1236879825592041 
Epoch: 2300 Error: 0.12353750318288803 
Epoch: 2400 Error: 0.12338823080062866 
Epoch: 2500 Error: 0.12323962152004242 
Epoch: 2600 E

Precision adalah suatu nilai dimana seluruh prediksi yang kita dapat adalah hasil yang relevan atau sesuai dengan target, tetapi tidak di ketahui apakah seluruh hasil yang relevan telah kita dapat atau tidak. sedangkan untuk recall adalah suatu nilai dimana seluruh nilai prediksi relevan telah kita ambil, namun tidak diketahui seberapa banyak data irrelevan yang telah kita ambil. Hal ini membuat nilai dari recall dan precision selalu bertolak belakang.Misalkan jika nilai recall naik maka nilai precision akan turun. Begitu juga sebaliknya

# Penjelasan model

jadi dari data di atas kita dapat bahwa precision,recall, dan accuracy yang saya dapat selalu berubah ubah. Hal ini dikarenakan weight dan bias yang saya gunakan adalah random sehingga hal ini menyebabkan inkosistensi dari hasil saya ini. Selama saya menjalankan koding ini saya mendapat hasil bahwa accuracy yang di dapat berkisar antara 40%-60%, untuk recall di dapat recall sebesar 35% sampai dengan 65%,dan untuk precision berkisar dari 50% sampai dengan 100%. jadi dapat kita di lihat bahwa data ini memiliki High Precision + high recall artinya model ini tidak akan mengmiss classify hasil positive dan juga hasil negative dari data data di atas.dapat juga di artikan bahwa model ini "Highly Depenable" dan juga dengan akurasi yang berada disekitar 50% membuat model ini mampu untuk memprediksi data dari dataset di atas untuk sesuai dengan targetnya.Kesimpulannya model ini sangat cocok untuk memprediksi dataset yang kita gunakan